ODIR Data

In [8]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import random

In [2]:
import os
path = r"ODIR-5K\ODIR-5K\Testing Images"
files = os.listdir(path)
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')
image_files = [f for f in files if f.lower().endswith(image_extensions)]
total_images = len(image_files)
print(f"Total number of images: {total_images}")

Total number of images: 1000


In [3]:
import os
path = r"ODIR-5K\ODIR-5K\Training Images"
files = os.listdir(path)
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')
image_files = [f for f in files if f.lower().endswith(image_extensions)]
total_images = len(image_files)
print(f"Total number of images: {total_images}")

Total number of images: 7000


In [7]:
import os
path = r"preprocessed_images"
files = os.listdir(path)
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')
image_files = [f for f in files if f.lower().endswith(image_extensions)]
total_images = len(image_files)
print(f"Total number of images: {total_images}")

Total number of images: 6392
